In [3]:
# Importamos las bibliotecas necesarias
import requests
import json
import datetime
import uuid
import copy
import time

print("Inicio del script - " + datetime.datetime.now().strftime("%H:%M:%S"))

# 1. Configuración
base_url = "http://localhost:8080"
username = "admin"
password = "admin"
dag_id = "publicacion_comprobante_pago_1"

# Parámetros configurables
num_peticiones = 20
pausa_entre_peticiones = 1

print(f"Configuración: {num_peticiones} peticiones con el mismo registro")
print(f"Pausa entre peticiones: {pausa_entre_peticiones} segundos")

# 2. Obtener token de autenticación
print("\nPASO 1: Obteniendo token de autenticación... - " + datetime.datetime.now().strftime("%H:%M:%S"))
auth_url = f"{base_url}/auth/token"
auth_data = {
    "username": username,
    "password": password
}

start_time = time.time()
response = requests.post(
    auth_url,
    headers={"Content-Type": "application/json"},
    data=json.dumps(auth_data)
)
auth_time = time.time() - start_time

print(f"Tiempo para autenticación: {auth_time:.2f} segundos")

if response.status_code in [200, 201]:
    token_data = response.json()
    token = token_data.get("access_token")
    print("Token obtenido correctamente")
else:
    print(f"Error al obtener token: {response.status_code}")
    print(response.text)
    exit(1)

# 3. Definir el registro único (factura base)
print("\nPASO 2: Definiendo la factura única... - " + datetime.datetime.now().strftime("%H:%M:%S"))
factura_base = {
    "NumeroFactura": "F001-2310255",
    "RucCliente": "PE20100055661",
    "FechaEmision": "2021-02-08T14:51:00-05:00",
    "NumeroGuia": "-",
    "NumeroOC": "-",
    "EstadoCliente": "Bloqueado",
    "FechaRecepcion": "2021-02-08T14:51:00-05:00",
    "FechaCreacion": "2021-02-08T14:51:00-05:00",
    "FechaEnvio": "2021-02-08T14:51:00-05:00",
    "DocumentoERP": "0200000027",
    "TipoFactura": "M",
    "LlaveERP": "15302000000080220211",
    "PeriodoPlazoPago": "30",
    "PlazoPago": "D",
    "FormaPago": "C030",
    "CondicionPago": "C030",
    "IdRegistroTipoEmision": "02",
    "Moneda": "PEN",
    "TipoComprobante": "01",
    "FechaVencimiento": "2021-03-30T14:51:00-05:00",
    "SubTotal": "4000.00",
    "TotalDescuento": "0.000",
    "Impuesto1": "720.00",
    "Impuesto2": "0",
    "Total": "4720.00",
    "Observaciones": "Factura única para pruebas de rendimiento",
    "FechaPago": "",
    "FechaProgramadaPago": "",
    "ItemFactura": [
        {
            "NumGuiaItem": "E001-53",
            "NumeroItemOC": "00010",
            "NumeroParte": "000000000000605665",
            "DescripcionItem": "SRV COBRANZA",
            "CantidadItem": "1.00",
            "UnidadMedidaItem": "BLD",
            "PrecioUnitario": "2000.00",
            "PrecioTotal": "2000.00",
            "NumeroOcItem": "4590023239",
            "TipoItem": "M"
        },
        {
            "NumGuiaItem": "E001-54",
            "NumeroItemOC": "00020",
            "NumeroParte": "000000000000605666",
            "DescripcionItem": "SRV COBRANZA",
            "CantidadItem": "1.00",
            "UnidadMedidaItem": "BLD",
            "PrecioUnitario": "2000.00",
            "PrecioTotal": "2000.00",
            "NumeroOcItem": "4590023240",
            "TipoItem": "M"
        }
    ],
    "Archivos": [
        {
            "nombre": "leeme2.txt",
            "archivo": "Descripción de archivo",
            "url": "https://storage.microsoft.co/demo2.pdf"
        }
    ]
}

# Crear la trama única que se usará en todas las peticiones
trama_unica = {
    "FACUPLOADMQ": {
        "RazonSocialProveedor": "RENA WARE DEL PERU S A",
        "RucProveedor": "PE20100131863",
        "Factura": [factura_base]
    }
}

# Estadísticas globales
peticiones_exitosas = 0
tiempo_total_envio = 0

# 4. Enviar el mismo registro múltiples veces
print("\nPASO 3: Enviando registros a Airflow... - " + datetime.datetime.now().strftime("%H:%M:%S"))

for i in range(num_peticiones):
    print(f"\n{'='*80}")
    print(f"PETICIÓN {i + 1} DE {num_peticiones} - " + datetime.datetime.now().strftime("%H:%M:%S"))
    print(f"{'='*80}")
    
    # Generar ID único y fecha para esta ejecución
    current_date = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
    run_id = f"proc_fact_peticion{i+1}_{uuid.uuid4().hex[:8]}"
    
    # Preparar datos para la petición
    data = {
        "dag_run_id": run_id,
        "logical_date": current_date,
        "conf": {
            "trama": trama_unica
        }
    }
    
    # Preparar headers
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    
    # Endpoint para ejecutar el DAG
    endpoint = f"{base_url}/api/v2/dags/{dag_id}/dagRuns"
    
    # Mostrar información de la petición
    print(f"URL: {endpoint}")
    print(f"ID de ejecución: {run_id}")
    print(f"Factura a enviar: {trama_unica['FACUPLOADMQ']['Factura'][0]['NumeroFactura']}")
    
    # Calcular tamaño aproximado de la petición
    request_size = len(json.dumps(data)) / 1024  # en KB
    print(f"Tamaño aproximado de la petición: {request_size:.2f} KB")
    
    # Enviar la petición y medir el tiempo
    start_time = time.time()
    
    try:
        response = requests.post(
            endpoint,
            headers=headers,
            data=json.dumps(data)
        )
        
        request_time = time.time() - start_time
        tiempo_total_envio += request_time
        
        print(f"Tiempo para enviar esta petición: {request_time:.2f} segundos")
        
        if response.status_code == 200:
            result = response.json()
            print(f"Respuesta exitosa: {result.get('dag_run_id')}")
            print(f"Estado: {result.get('state')}")
            peticiones_exitosas += 1
        else:
            print(f"Error al ejecutar el DAG: {response.status_code}")
            print(response.text)
    except Exception as e:
        print(f"Error en la solicitud: {str(e)}")
    
    # Pausa entre peticiones (excepto después de la última)
    if i < num_peticiones - 1:
        print(f"Esperando {pausa_entre_peticiones} segundos antes de la siguiente petición...")
        time.sleep(pausa_entre_peticiones)

# Resumen final
print("\n" + "="*50)
print("RESUMEN GLOBAL:")
print("="*50)
print(f"Tiempo de autenticación: {auth_time:.2f} segundos")
print(f"Tiempo total de envío: {tiempo_total_envio:.2f} segundos")
print(f"Tiempo promedio por petición: {tiempo_total_envio/num_peticiones:.2f} segundos")
print(f"Tiempo total del script: {auth_time + tiempo_total_envio:.2f} segundos")
print(f"Total peticiones realizadas: {num_peticiones}")
print(f"Peticiones exitosas: {peticiones_exitosas}")
print(f"Peticiones fallidas: {num_peticiones - peticiones_exitosas}")
print("\nFin del script - " + datetime.datetime.now().strftime("%H:%M:%S"))

Inicio del script - 08:17:10
Configuración: 20 peticiones con el mismo registro
Pausa entre peticiones: 1 segundos

PASO 1: Obteniendo token de autenticación... - 08:17:10
Tiempo para autenticación: 1.85 segundos
Token obtenido correctamente

PASO 2: Definiendo la factura única... - 08:17:12

PASO 3: Enviando registros a Airflow... - 08:17:12

PETICIÓN 1 DE 20 - 08:17:12
URL: http://localhost:8080/api/v2/dags/publicacion_comprobante_pago_1/dagRuns
ID de ejecución: proc_fact_peticion1_e75b6f48
Factura a enviar: F001-2310255
Tamaño aproximado de la petición: 1.72 KB


C:\Users\USER\AppData\Local\Temp\ipykernel_6608\2911763838.py:139: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_date = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"


Tiempo para enviar esta petición: 0.78 segundos
Respuesta exitosa: proc_fact_peticion1_e75b6f48
Estado: queued
Esperando 1 segundos antes de la siguiente petición...

PETICIÓN 2 DE 20 - 08:17:14
URL: http://localhost:8080/api/v2/dags/publicacion_comprobante_pago_1/dagRuns
ID de ejecución: proc_fact_peticion2_9cf0013a
Factura a enviar: F001-2310255
Tamaño aproximado de la petición: 1.72 KB
Tiempo para enviar esta petición: 0.60 segundos
Respuesta exitosa: proc_fact_peticion2_9cf0013a
Estado: queued
Esperando 1 segundos antes de la siguiente petición...

PETICIÓN 3 DE 20 - 08:17:16
URL: http://localhost:8080/api/v2/dags/publicacion_comprobante_pago_1/dagRuns
ID de ejecución: proc_fact_peticion3_7164afc6
Factura a enviar: F001-2310255
Tamaño aproximado de la petición: 1.72 KB
Tiempo para enviar esta petición: 0.70 segundos
Respuesta exitosa: proc_fact_peticion3_7164afc6
Estado: queued
Esperando 1 segundos antes de la siguiente petición...

PETICIÓN 4 DE 20 - 08:17:17
URL: http://localhost